In [16]:
import os
import sqlite3
import pandas as pd

sql_query = "select * from data"

# with open(your_sql_filepath, "r") as f:
#     sql_query = f.read()
conn = sqlite3.connect(os.path.join("assets", "task2.db"))
df = pd.read_sql(sql_query, conn)
df

DatabaseError: Execution failed on sql 'select * from data': no such table: data

In [20]:
import pandas as pd
import sqlite3

# Create a connection to the in-memory SQLite database
conn = sqlite3.connect(os.path.join("assets", "task2.db"))

# Load the data using the provided SQL query
query = """
SELECT 
    c.periodId,
    c.institutionId,
    c.reportedCurrencyISOCode,
    c.tcPrimarySectorId AS companyPrimarySectorId,
    c.fiscalYear AS companyFiscalYear,
    c.periodEndDate AS companyPeriodEndDate,
    s.sectorPeriodId,
    s.tcPrimarySectorId AS sectorPrimarySectorId,
    s.tcSectorId,
    s.fiscalYear AS sectorFiscalYear,
    s.periodEndDate AS sectorPeriodEndDate,
    di.dataItemId,
    di.dataItemName,
    di.dataItemDefinition,
    di.magnitude,
    di.isText,
    e.tcEnvType,
    pd.dataItemValue AS companyDataItemValue,
    ptd.dataItemValueText AS companyDataItemValueText,
    spd.dataItemValue AS sectorDataItemValue,
    sptd.dataItemValueText AS sectorDataItemValueText
FROM 
    tcPeriod c
    LEFT JOIN tcPeriodData pd ON c.periodId = pd.periodId
    LEFT JOIN tcPeriodTextData ptd ON c.periodId = ptd.periodId
    LEFT JOIN tcSectorPeriod s ON c.institutionId = s.institutionId 
        AND c.reportedCurrencyISOCode = s.reportedCurrencyISOCode 
        AND c.tcPrimarySectorId = s.tcPrimarySectorId
        AND c.fiscalYear = s.fiscalYear
        AND c.periodEndDate = s.periodEndDate
    LEFT JOIN tcSectorPeriodData spd ON s.sectorPeriodId = spd.sectorPeriodId
    LEFT JOIN tcSectorPeriodTextData sptd ON s.sectorPeriodId = sptd.sectorPeriodId
    LEFT JOIN tcDataItem di ON pd.dataItemId = di.dataItemId OR ptd.dataItemId = di.dataItemId
        OR spd.dataItemId = di.dataItemId OR sptd.dataItemId = di.dataItemId
    LEFT JOIN tcDataItemToEnvType dite ON di.dataItemId = dite.dataItemId
    LEFT JOIN tcEnvType e ON dite.tcEnvTypeId = e.tcEnvTypeId
    LEFT JOIN tcSector sec ON s.tcSectorId = sec.tcSectorId;

"""
df = pd.read_sql_query(query, conn)
df

,periodid,institutionid,reportedcurrencyisocode,companyPrimarySectorId,companyFiscalYear,companyPeriodEndDate,sectorperiodid,sectorPrimarySectorId,tcsectorid,sectorFiscalYear,...,dataitemid,dataitemname,dataitemdefinition,magnitude,istext,tcenvtype,companyDataItemValue,companyDataItemValueText,sectorDataItemValue,sectorDataItemValueText
0,EA004A96-4BF6-42A3-A534-0000A9ACC61D,0,USD,336111,2018,2018-03-31 00:00:00,None,None,None,NaN,...,319380.0,Absolute: Air Pollutants Direct Cost,External cost of pollutants released to air by...,$M,0.0,Trucost Environmental Data,0.642925,Estimate used instead of disclosure - data doe...,NaN,None
1,EA004A96-4BF6-42A3-A534-0000A9ACC61D,0,USD,336111,2018,2018-03-31 00:00:00,None,None,None,NaN,...,319403.0,Scope 1 Carbon Disclosure,Scope 1 Carbon Disclosure,None,1.0,Trucost Environmental Data,0.642925,Estimate used instead of disclosure - data doe...,NaN,None
2,EA004A96-4BF6-42A3-A534-0000A9ACC61D,0,USD,336111,2018,2018-03-31 00:00:00,None,None,None,NaN,...,319380.0,Absolute: Air Pollutants Direct Cost,External cost of pollutants released to air by...,$M,0.0,Trucost Environmental Data,0.642925,Aug 5 2019 12:19PM,NaN,None
3,EA004A96-4BF6-42A3-A534-0000A9ACC61D,0,USD,336111,2018,2018-03-31 00:00:00,None,None,None,NaN,...,323615.0,Effective date of Environmental Cost Calculation,Date on which environmental data was last modi...,None,1.0,Trucost Environmental Data,0.642925,Aug 5 2019 12:19PM,NaN,None
4,EA004A96-4BF6-42A3-A534-0000A9ACC61D,0,USD,336111,2018,2018-03-31 00:00:00,None,None,None,NaN,...,319380.0,Absolute: Air Pollutants Direct Cost,External cost of pollutants released to air by...,$M,0.0,Trucost Environmental Data,0.642925,Aug 5 2019 12:19PM,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124344,D001ED45-4042-4F0D-B159-0D0F4561AC7D,4170,USD,33351A,2006,2006-11-30 00:00:00,None,None,None,NaN,...,NaN,None,None,None,NaN,None,NaN,None,NaN,None
124345,22F2E8D3-5F13-48CD-A3BB-60C3D8945244,2103,USD,33441A,2022,2022-03-31 00:00:00,None,None,None,NaN,...,NaN,None,None,None,NaN,None,NaN,None,NaN,None
124346,A81D30E3-1B2D-4C37-9127-0D0FDBAB292D,4171,USD,52A000,2009,2009-12-31 00:00:00,None,None,None,NaN,...,NaN,None,None,None,NaN,None,NaN,None,NaN,None
124347,5DE536AB-816D-4290-A9FD-0D0FF0BCF316,4172,USD,311210,2010,2010-12-31 00:00:00,None,None,None,NaN,...,NaN,None,None,None,NaN,None,NaN,None,NaN,None


In [26]:

# Transform the dataframe by pivoting on `dataItemName`
pivot_df = df.pivot_table(
    index=['periodid', 'institutionid', 'reportedcurrencyisocode', 'companyPrimarySectorId',
           'companyFiscalYear', 'companyPeriodEndDate', 'sectorperiodid', 'sectorPrimarySectorId',
           'tcsectorid', 'sectorFiscalYear', 'sectorPeriodEndDate', 'tcenvtype'],
    columns='dataitemname',
    values=['companyDataItemValue', 'companyDataItemValueText', 'sectorDataItemValue', 'sectorDataItemValueText']
).reset_index()

# Save the transformed dataframe to a CSV file
csv_file_path = '/mnt/data/transformed_data.csv'
pivot_df.to_csv(csv_file_path, index=False)

# Save the SQL query to a .sql file
sql_file_path = '/mnt/data/merge_and_transform.sql'
with open(sql_file_path, 'w') as file:
    file.write(query)


TypeError: agg function failed [how->mean,dtype->object]